In [1]:
import requests
import json
import pandas as pd

In [9]:
files_endpt = "https://api.gdc.cancer.gov/files"

# The 'fields' parameter is passed as a comma-separated string of single names.
fields = [
    "file_name",
    #"analysis.workflow_type",
    #"experimental_strategy",
    "cases.project.primary_site",
    "cases.project.disease_type",
    "cases.diagnoses.primary_diagnosis",
    "cases.submitter_id",
    "cases.samples.portions.analytes.aliquots.submitter_id",
    "cases.diagnoses.tumor_stage",
    "cases.diagnoses.tumor_grade",
    "cases.diagnoses.progression_or_recurrence"
    "cases.diagnoses.vital_status",
    "cases.diagnoses.days_to_birth",
    "cases.diagnoses.days_to_death",
    "cases.diagnoses.morphology",
    "cases.diagnoses.tissue_or_organ_of_origin",
    "cases.samples.longest_dimension"
    #"cases.project.project_id"
]

fields = ','.join(fields)

In [ ]:
#miRNA filters
filters = {
    "op": "and",
    "content":[
        {
        "op": "in",
        "content":{
            "field": "files.data_type",
            "value": ["miRNA Expression Quantification"]
            }
        },
          {
        "op": "in",
        "content":{
               "field": "files.experimental_strategy",
                "value": ["miRNA-Seq"]
                }
        },
        {
        "op": "in",
        "content":{
            "field": "files.data_format",
            "value": ["TXT"]
            }
        },
        {
        "op": "in",
        "content":{
            "field": "cases.primary_site",
            "value": ['Brain', 'Breast', 'Kidney', 'Lung', 'Thyroid', 'Uterus',
       'Prostate', 'Ovary', 'Lymph Nodes', 'Soft Tissue', 'Esophagus',
       'Stomach', 'Bone Marrow']
            }
        },
        {
        "op": "in",
        "content":{
            "field": "cases.project.program.name",
            "value": ["TCGA"]
            }
        }
        
    ]
}

In [10]:
# Tissue project filters
filters = {
    "op": "and",
    "content":[
        {
        "op": "in",
        "content":{
            "field": "files.data_type",
            "value": ["Gene Expression Quantification"]
            }
        },
        {
        "op": "in",
        "content":{
               "field": "files.analysis.workflow_type",
                "value": ["HTSeq - FPKM"]
                }
        },
        {
        "op": "in",
        "content":{
            "field": "files.data_format",
            "value": ["TXT"]
            }
        },
        {
        "op": "in",
        "content":{
            "field": "cases.project.project_id",
            "value": ['TCGA-COAD']
            }
        }
        
    ]
}

In [3]:
# all files RNA-seq
filters = {
    "op": "and",
    "content":[
        {
        "op": "in",
        "content":{
            "field": "files.data_type",
            "value": ["Gene Expression Quantification"]
            }
        },
        {
        "op": "in",
        "content":{
               "field": "files.analysis.workflow_type",
                "value": ["HTSeq - FPKM"]
                }
        },
        {
        "op": "in",
        "content":{
            "field": "files.data_format",
            "value": ["TXT"]
            }
        },
        {
        "op": "in",
        "content":{
            "field": "cases.project.program.name",
            "value": ["TCGA"]
            }
        }
        
    ]
}

In [11]:
# A POST is used, so the filter parameters can be passed directly as a Dict object.
##remove manifest to have file fields
params = {
   "return_type": "manifest",
    "filters": filters,
    "fields": fields,
    "format": "TSV",
    "size": "15000"
    }

In [12]:
# The parameters are passed to 'json' rather than 'params' in this case
response = requests.post(files_endpt, headers = {"Content-Type": "application/json"}, json = params)

#print(response.content.decode("utf-8"))

In [13]:
len(response.content.decode("utf-8"))

70362

In [14]:
with open("manifest.txt","w") as manifest:
    manifest.write(response.content.decode("utf-8"))
    manifest.close()

## Files

In [15]:
params = {
    "filters": filters,
    "fields": fields,
    "format": "TSV",
    "size": "15000"
    }
response = requests.post(files_endpt, headers = {"Content-Type": "application/json"}, json = params)
with open("files.txt","w") as files:
    files.write(response.content.decode("utf-8"))

In [16]:
df_files = pd.read_csv("files.txt", sep='\t')
df_files.drop("id", axis=1, inplace=True)
df_files.set_index("file_name", inplace=True)
#df_files.columns=['primary_site','tcga_id','disease_type','primary_diagnosis']
#df_files=df_files.reindex(columns=['primary_site','disease_type','primary_diagnosis','case_id'], copy=False)
df_files.head()

,cases.0.diagnoses.0.morphology,cases.0.diagnoses.0.primary_diagnosis,cases.0.diagnoses.0.tissue_or_organ_of_origin,cases.0.diagnoses.0.tumor_grade,cases.0.diagnoses.0.tumor_stage,cases.0.project.disease_type,cases.0.project.primary_site,cases.0.samples.0.longest_dimension,cases.0.samples.0.portions.0.analytes.0.aliquots.0.submitter_id,cases.0.submitter_id
file_name,,,,,,,,,,
020aa019-a3a4-4055-92ee-be824a597501.FPKM.txt.gz,8140/3,"Adenocarcinoma, NOS",Sigmoid colon,not reported,stage iia,Colon Adenocarcinoma,Colorectal,0.7,TCGA-CM-6165-01A-11R-1653-07,TCGA-CM-6165
00106523-5b1d-44ad-a9f1-7d84db08722c.FPKM.txt.gz,8140/3,"Adenocarcinoma, NOS",Cecum,not reported,stage iv,Colon Adenocarcinoma,Colorectal,0.6,TCGA-G4-6294-01A-11R-1774-07,TCGA-G4-6294
3eb03db7-efc1-4f20-bbc9-61551905eb9b.FPKM.txt.gz,8140/3,"Adenocarcinoma, NOS",Cecum,not reported,stage ii,Colon Adenocarcinoma,Colorectal,NaN,TCGA-AA-A022-01A-21R-A16W-07,TCGA-AA-A022
ddc5dfb2-e7e4-4159-930e-120c7d2a8640.FPKM.txt.gz,8140/3,"Adenocarcinoma, NOS",Ascending colon,not reported,not reported,Colon Adenocarcinoma,Colorectal,0.3,TCGA-AD-6963-01A-11R-1928-07,TCGA-AD-6963
29dab10a-a0a0-4dbd-b574-97c22b70a3f1.FPKM.txt.gz,8140/3,"Adenocarcinoma, NOS","Colon, NOS",not reported,stage i,Colon Adenocarcinoma,Colorectal,NaN,TCGA-A6-2684-01C-08R-A277-07,TCGA-A6-2684


In [17]:
df_files.to_csv("files.dat", header=True)

In [ ]:
params = {
    "filters": json.dumps(filters),
    "fields": "primary_site,disease_type,files.cases.demographic.vital_status,diagnoses.vital_status",
    "format": "TSV",
    "size": "10"
    }
response = requests.get("https://api.gdc.cancer.gov/cases", headers = {"Content-Type": "application/json"}, params = params)
with open("files.txt","w") as files:
    files.write(response.content.decode("utf-8"))
    files.close()